In [6]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Initialize spark driver
findspark.init()
spark = SparkSession.builder.master("local").appName("FSTExample").getOrCreate()

# TO read the whole XML file as a single string 
file_rdd = spark.read.text('./Inputs/persons.xml', wholetext=True).rdd

# Function to parse the file and extract data
ELEMENTS_TO_EXTRACT = ['firstname', 'middlename', 'lastname', 'dob_year', 'dob_month', 'salary', 'gender']
def parse_xml(rdd):
    """
    This function is used to read an xml string from rdd, parse and extract the elements,
    then return a list of lists.
    """
    results = []
    root = ET.fromstring(rdd[0])

    for p in root.findall('person'):
        rec = []
        rec.append(p.attrib['id'])

        for e in ELEMENTS_TO_EXTRACT:
            if p.find(e) is None:
                rec.append(None)
                continue
            value = p.find(e).text
            if e == 'salary':
                value = float(value)
            rec.append(value)
        results.append(rec)
    return results
  
# To view the data frame
records_rdd = file_rdd.flatMap(parse_xml)
persons = records_rdd.toDF(("id", "firstName", "middleName", "lastName", "dob_year", "dob_month", "salary", "gender"))
persons.show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 11) (IBM-PF42SQ40 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "c:\Users\004OVH744\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1247, in main
  File "c:\Users\004OVH744\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1239, in process
  File "c:\Users\004OVH744\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\004OVH744\AppData\Local\Programs\Py

In [5]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Initialize spark driver
findspark.init()
spark = SparkSession.builder.master("local").appName("FSTExample").getOrCreate()

# To display the schema of the dataframe
df = spark.read.option('multiline', 'true').json('./Inputs/zipcodes.json')
df.printSchema()

# To select data from the table
zc = df.select(df.RecordNumber, df.Zipcode, df.ZipCodeType, df.City, df.State, df.Country, df.WorldRegion, df.Decommisioned)
zc.show()

# To filter records
zc.filter((df.State == 'PR') | (df.State == 'TX')).show()
zc.filter((df.State == 'PR') & (df.Zipcode == 709)).show()

# To sort data
zc.orderBy(df.Zipcode.asc()).show()

# To perform aggregate operations
zc.groupBy('State').count().orderBy('count', ascending=False).show()

root
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Decommisioned: boolean (nullable = true)
 |-- RecordNumber: long (nullable = true)
 |-- State: string (nullable = true)
 |-- WorldRegion: string (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- Zipcode: long (nullable = true)

+------------+-------+-----------+-------------------+-----+-------+-----------+-------------+
|RecordNumber|Zipcode|ZipCodeType|               City|State|Country|WorldRegion|Decommisioned|
+------------+-------+-----------+-------------------+-----+-------+-----------+-------------+
|           1|    704|   STANDARD|        PARC PARQUE|   PR|     US|         NA|        false|
|           2|    704|   STANDARD|PASEO COSTA DEL SUR|   PR|     US|         NA|        false|
|          10|    709|   STANDARD|       BDA SAN LUIS|   PR|     US|         NA|        false|
|       61391|  76166|     UNIQUE|  CINGULAR WIRELESS|   TX|     US|         NA|        false|
|     

In [3]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Initialize spark driver
findspark.init()
spark = SparkSession.builder.master("local").appName("FSTExample").getOrCreate()

# Find all Thriller movies with a Metascore greater than 75
inputFile = spark.read.option("header", True).csv("./Inputs/movies.csv")
inputFile.select("Title", "Genre", "Metascore").filter(("Metascore > 75 and Genre like '%Thriller%'")).sort(desc("Metascore")).show()

+--------------------+--------------------+---------+
|               Title|               Genre|Metascore|
+--------------------+--------------------+---------+
|             Gravity|Drama,Sci-Fi,Thri...|       96|
|    Zero Dark Thirty|Drama,History,Thr...|       95|
|    Megan Is Missing|Drama,Horror,Thri...|       94|
|     The Hurt Locker|Drama,History,Thr...|       94|
|No Country for Ol...|Crime,Drama,Thriller|       91|
|                Elle|Crime,Drama,Thriller|       89|
| The Lives of Others|      Drama,Thriller|       89|
|  Hell or High Water|Crime,Drama,Thriller|       88|
|        The Departed|Crime,Drama,Thriller|       85|
|The Bourne Ultimatum|Action,Mystery,Th...|       85|
|Tinker Tailor Sol...|Drama,Mystery,Thr...|       85|
|     Children of Men|Drama,Sci-Fi,Thri...|       84|
|    Captain Phillips|Biography,Drama,T...|       83|
|The Siege of Jado...|Action,Drama,Thri...|       83|
|             Skyfall|Action,Adventure,...|       81|
|     Bridge of Spies|Drama,

In [2]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Initialize spark driver
findspark.init()
spark = SparkSession.builder.master("local").appName("FSTExample").getOrCreate()

# Find the word count from a text file
inputFile = spark.read.text("./testfile.txt")
inputFile.select(explode(split(inputFile.value, '\s+')).alias('word')).groupBy('word').count().show()

+------+-----+
|  word|count|
+------+-----+
|  name| 1008|
|    is| 1008|
|Rajesh| 1008|
|    my| 1008|
|      | 1029|
+------+-----+

